<a href="https://colab.research.google.com/github/joosk3R/jskRprac/blob/main/dacon_e_e.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
train = pd.read_csv("/content/train.csv")

In [4]:
test = pd.read_csv("/content/test.csv")
sample = pd.read_csv("/content/sample_submission.csv")

In [5]:
df = pd.DataFrame()

In [7]:
df["date"] = train['일시'].apply(lambda x : int(x[0:8]))

In [8]:
import pandas as pd

# Assuming you have a DataFrame called 'df' with a column named 'date' containing date values
# Convert the 'date' column to datetime type
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')

# Create a new column 'is_weekend' indicating weekends as 1 and weekdays as 0
df['is_weekend'] = df['date'].dt.dayofweek // 5

# Output the modified DataFrame
print(df)

             date  is_weekend
0      2022-06-01           0
1      2022-06-01           0
2      2022-06-01           0
3      2022-06-01           0
4      2022-06-01           0
...           ...         ...
203995 2022-08-24           0
203996 2022-08-24           0
203997 2022-08-24           0
203998 2022-08-24           0
203999 2022-08-24           0

[204000 rows x 2 columns]


In [9]:
bd = pd.read_csv("/content/building_info.csv")

In [10]:
train["강수량(mm)"] = train["강수량(mm)"].fillna(0)
train["일사(MJ/m2)"] = train["일사(MJ/m2)"].fillna(0)
train["일조(hr)"] = train["일조(hr)"].fillna(0)
train["풍속(m/s)"] = train["풍속(m/s)"].fillna(method='bfill')
train["습도(%)"] = train["습도(%)"].fillna(method='bfill')

In [11]:
train['month'] = train['일시'].apply(lambda x : int(x[4:6]))
train['day'] = train['일시'].apply(lambda x : int(x[6:8]))
train['time'] = train['일시'].apply(lambda x : int(x[9:11]))

In [12]:
test['month'] = test['일시'].apply(lambda x : int(x[4:6]))
test['day'] = test['일시'].apply(lambda x : int(x[6:8]))
test['time'] = test['일시'].apply(lambda x : int(x[9:11]))

In [13]:
train_x = train.drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)'])
train_y = train['전력소비량(kWh)']

In [14]:
test_x = test.drop(columns=['num_date_time', '일시'])


In [15]:
train_x['is_weekend']  =df["is_weekend"]
train_x['is_working_hour'] = train_x['time'].apply(lambda x: 1 if 7 <= x <= 21 else 0)

test_x['is_weekend']  =df["is_weekend"]
test_x['is_working_hour'] = test_x['time'].apply(lambda x: 1 if 7 <= x <= 21 else 0)


In [16]:
import pandas as pd

# Assuming you have already loaded your data into df1 and df2

# Merge df1 and df2 on "Building Number" column
train_df = train_x.merge(bd, on="건물번호", how="inner")
test_df = test_x.merge(bd, on="건물번호", how="inner")


In [17]:
y = train_df["전력소비량(kWh)"]


In [18]:
train_df = train_df.drop(columns=["전력소비량(kWh)", 'month','day','time'])
test_df = test_df.drop(columns=[ 'month','day','time'])

In [19]:
train_dum = pd.get_dummies(train_df, columns = ['is_working_hour','is_weekend','건물유형'])
test_dum = pd.get_dummies(test_df, columns = ['is_working_hour','is_weekend','건물유형'])
train_dum.head()

,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%),연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW),...,건물유형_대학교,건물유형_데이터센터,건물유형_백화점및아울렛,건물유형_병원,건물유형_상용,건물유형_아파트,건물유형_연구소,건물유형_지식산업센터,건물유형_할인마트,건물유형_호텔및리조트
0,1,18.6,0.0,0.9,42.0,110634.0,39570.0,-,-,-,...,0,0,0,0,0,0,0,0,0,0
1,1,18.0,0.0,1.1,45.0,110634.0,39570.0,-,-,-,...,0,0,0,0,0,0,0,0,0,0
2,1,17.7,0.0,1.5,45.0,110634.0,39570.0,-,-,-,...,0,0,0,0,0,0,0,0,0,0
3,1,16.7,0.0,1.4,48.0,110634.0,39570.0,-,-,-,...,0,0,0,0,0,0,0,0,0,0
4,1,18.4,0.0,2.8,43.0,110634.0,39570.0,-,-,-,...,0,0,0,0,0,0,0,0,0,0


In [20]:
train_X = train_dum.drop(columns=['건물번호','태양광용량(kW)', 'ESS저장용량(kWh)','강수량(mm)',
       'PCS용량(kW)',])
test_X = test_dum.drop(columns=['건물번호','태양광용량(kW)', 'ESS저장용량(kWh)','강수량(mm)',
       'PCS용량(kW)',])

In [21]:
from sklearn.preprocessing import Normalizer # 정규화

norm = Normalizer()
X_train_scaled = norm.fit_transform(train_X.iloc[:,0:5])
X_test_scaled = norm.transform(test_X.iloc[:,0:5])

In [22]:
train_scaled = pd.DataFrame(data = X_train_scaled)
test_scaled = pd.DataFrame(data = X_test_scaled)

In [23]:
train_xx = pd.concat([train_scaled,train_X.iloc[:,5:]],axis=1)
test_xx = pd.concat([test_scaled,test_X.iloc[:,5:]],axis=1)

In [79]:
train_xx

,0,1,2,3,4,is_working_hour_0,is_working_hour_1,is_weekend_0,is_weekend_1,건물유형_건물기타,...,건물유형_대학교,건물유형_데이터센터,건물유형_백화점및아울렛,건물유형_병원,건물유형_상용,건물유형_아파트,건물유형_연구소,건물유형_지식산업센터,건물유형_할인마트,건물유형_호텔및리조트
0,0.000158,0.000008,0.000357,0.941586,0.336773,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0.000153,0.000009,0.000383,0.941586,0.336773,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0.000151,0.000013,0.000383,0.941586,0.336773,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0.000142,0.000012,0.000409,0.941586,0.336773,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0.000157,0.000024,0.000366,0.941586,0.336773,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203995,0.000330,0.000013,0.001227,0.820659,0.571417,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
203996,0.000320,0.000019,0.001227,0.820659,0.571417,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
203997,0.000304,0.000014,0.001313,0.820659,0.571417,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
203998,0.000300,0.000004,0.001342,0.820659,0.571417,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [80]:
test_xx

,0,1,2,3,4,is_working_hour_0,is_working_hour_1,is_weekend_0,is_weekend_1,건물유형_건물기타,...,건물유형_대학교,건물유형_데이터센터,건물유형_백화점및아울렛,건물유형_병원,건물유형_상용,건물유형_아파트,건물유형_연구소,건물유형_지식산업센터,건물유형_할인마트,건물유형_호텔및리조트
0,0.000200,0.000019,0.000613,0.941586,0.336773,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0.000196,0.000008,0.000613,0.941586,0.336773,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0.000193,0.000013,0.000638,0.941586,0.336773,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0.000188,0.000011,0.000664,0.941586,0.336773,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0.000186,0.000009,0.000655,0.941586,0.336773,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,0.000321,0.000013,0.001199,0.820659,0.571417,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
16796,0.000295,0.000006,0.001356,0.820659,0.571417,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
16797,0.000288,0.000006,0.001399,0.820659,0.571417,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
16798,0.000287,0.000016,0.001384,0.820659,0.571417,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [81]:
y

0         1085.28
1         1047.36
2          974.88
3          953.76
4          986.40
           ...   
203995     881.04
203996     798.96
203997     825.12
203998     640.08
203999     540.24
Name: 전력소비량(kWh), Length: 204000, dtype: float64

In [ ]:
!pip install autogluon

In [28]:
train_x = pd.concat([train_xx,y],axis=1)
test_x = test_xx

In [29]:
test_x

,0,1,2,3,4,is_working_hour_0,is_working_hour_1,is_weekend_0,is_weekend_1,건물유형_건물기타,...,건물유형_대학교,건물유형_데이터센터,건물유형_백화점및아울렛,건물유형_병원,건물유형_상용,건물유형_아파트,건물유형_연구소,건물유형_지식산업센터,건물유형_할인마트,건물유형_호텔및리조트
0,0.000200,0.000019,0.000613,0.941586,0.336773,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0.000196,0.000008,0.000613,0.941586,0.336773,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0.000193,0.000013,0.000638,0.941586,0.336773,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0.000188,0.000011,0.000664,0.941586,0.336773,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,0.000186,0.000009,0.000655,0.941586,0.336773,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16795,0.000321,0.000013,0.001199,0.820659,0.571417,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
16796,0.000295,0.000006,0.001356,0.820659,0.571417,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
16797,0.000288,0.000006,0.001399,0.820659,0.571417,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
16798,0.000287,0.000016,0.001384,0.820659,0.571417,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1


In [24]:
import os
import pandas as pd
import random
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
import autogluon.core as ag

import warnings
warnings.filterwarnings("ignore")

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(69)


In [30]:
train_data = TabularDataset(train_x)
test_data = TabularDataset(test_x)

In [43]:
label = '전력소비량(kWh)'
eval_metric = 'rmse'
time_limit = 3600*2

In [44]:
excluded_model_types = ['CAT']
predictor = TabularPredictor(
    label=label, problem_type='regression', eval_metric=eval_metric
).fit(train_data,
      presets='best_quality',
      num_stack_levels=3,
      excluded_model_types = excluded_model_types,
      time_limit=time_limit, num_gpus=1)

No path specified. Models will be saved in: "AutogluonModels/ag-20230719_031957/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 7200s
AutoGluon will save models to "AutogluonModels/ag-20230719_031957/"
AutoGluon Version:  0.8.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Jun 9 10:57:30 UTC 2023
Disk Space Avail:   146.04 GB / 179.07 GB (81.6%)
Train Data Rows:    204000
Train Data Columns: 21
Label Column: 전력소비량(kWh)
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    24030.93 MB
	Train Data (Original)  Memory Usage: 11.42 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stag

In [45]:
model_to_use = predictor.get_model_best()
model_pred = predictor.predict(test_data, model=model_to_use)

submission = pd.read_csv('/content/sample_submission.csv')

submission["전화해지여부"] = model_pred
submission.to_csv('./Gluon_submission.csv', index=False, encoding="utf-8")

autogluon결과 전체 데이터를 사용하면 예측력이 떨어짐 시계열 형태로 각각의 건물마다 데이터를 추정해주는것이 좋을 것 같음